In [1]:
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import SubsetRandomSampler

import os
import argparse
import random
from utils import progress_bar
from model_wrn import wrn

In [2]:
# Data preprocessing 
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

==> Preparing data..


In [3]:
# Parameters 
# parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
# parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
# parser.add_argument('--resume', '-r', action='store_true', help='resume from checkpoint')
# parser.add_argument('--total_epoch', default=200,type=int)
# parser.add_argument('--model_dir',default='./checkpoint_wrn/')
# args = parser.parse_args()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [4]:
# Load training images of CIFAR-10 twice: one for training, one for validation, with different preprocessing function
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
val_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# Code to resume training process, ignored
# if args.resume:
#     # Load checkpoint.
#     print('==> Resuming from checkpoint..')
#     assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
#     checkpoint = torch.load('./checkpoint/ckpt.pth')
#     net.load_state_dict(checkpoint['net'])
#     best_acc = checkpoint['acc']
#     start_epoch = checkpoint['epoch']

In [6]:
###Learning Rate Schedule###
def adjust_learning_rate(optimizer, epoch):
    if(epoch<61):
        lr=0.1
    elif(epoch<121):
        lr=0.02
    elif(epoch<161):
        lr=0.004
    elif(epoch<=200):
        lr=0.0008
    #print(lr)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [7]:
###Training Code###
def train(epoch,dataLoader,model):
    adjust_learning_rate(optimizer, epoch)

    print('Epoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(dataLoader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        # progress_bar(batch_idx, len(dataLoader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
        #     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

In [8]:
###Testing Code for model selection###
def test(epoch,dataLoader,model):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    batch_num=0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataLoader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            batch_num+=1

            # progress_bar(batch_idx, len(dataLoader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            #     % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))


    # Save best model according to the performance on validation set
    acc = 100.*correct/total
    print('Loss: %.3f | Acc: %.3f%% (%d/%d)' % (test_loss/(batch_num), 100.*correct/total, correct, total))
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint-no-cross/' + 'ckpt{}.pth'.format(0))
        best_acc = acc

In [9]:
# 5-fold cross-validation: Parameter settings, shuffle the data order
# model_num = 5
# indices = list(range(len(train_dataset)))
# random.shuffle(indices)
# val_size = int(float(1/model_num)*len(train_dataset))

In [10]:
import torchsummary

net = wrn(
          num_classes=10,
          depth=16,
          widen_factor=5,
          dropRate=0.3,
        )
net.cuda()
torchsummary.summary(net, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
            Conv2d-4           [-1, 80, 32, 32]          11,520
       BatchNorm2d-5           [-1, 80, 32, 32]             160
              ReLU-6           [-1, 80, 32, 32]               0
            Conv2d-7           [-1, 80, 32, 32]          57,600
            Conv2d-8           [-1, 80, 32, 32]           1,280
        BasicBlock-9           [-1, 80, 32, 32]               0
      BatchNorm2d-10           [-1, 80, 32, 32]             160
             ReLU-11           [-1, 80, 32, 32]               0
           Conv2d-12           [-1, 80, 32, 32]          57,600
      BatchNorm2d-13           [-1, 80, 32, 32]             160
             ReLU-14           [-1, 80,

In [11]:
# for model_index in range(model_num):
    #Split Dataset
#     val_indices = indices[val_size * model_index: val_size*(model_index+1)]
#     train_indices = indices[:val_size*model_index]+indices[val_size* \
#             (model_index+1):]         
#     train_sampler = SubsetRandomSampler(train_indices)
#     val_sampler = SubsetRandomSampler(val_indices)
    
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=False, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=100, shuffle=False, num_workers=2)


#Initialize Baseline Model
print('================> Building model {}..'.format(0))
net =wrn(
      num_classes=10,
      depth=16,
      widen_factor=5,
      dropRate=0.3,
    )
net = net.to(device)

if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True


#Initialize Optimizer
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)


#training and testing
for epoch in range(start_epoch, start_epoch + 200):
    train(epoch,train_loader,net)
    test(epoch,val_loader,net)

================> Building model 0..
Epoch: 0
Loss: 1.507 | Acc: 49.230% (4923/10000)
Saving..
Epoch: 1
Loss: 0.929 | Acc: 67.560% (6756/10000)
Saving..
Epoch: 2
Loss: 0.907 | Acc: 69.590% (6959/10000)
Saving..
Epoch: 3
Loss: 0.824 | Acc: 72.860% (7286/10000)
Saving..
Epoch: 4
Loss: 0.714 | Acc: 76.170% (7617/10000)
Saving..
Epoch: 5
Loss: 0.749 | Acc: 75.620% (7562/10000)
Epoch: 6
Loss: 0.700 | Acc: 77.210% (7721/10000)
Saving..
Epoch: 7
Loss: 0.696 | Acc: 78.000% (7800/10000)
Saving..
Epoch: 8
Loss: 0.675 | Acc: 78.130% (7813/10000)
Saving..
Epoch: 9
Loss: 0.837 | Acc: 75.110% (7511/10000)
Epoch: 10
Loss: 0.755 | Acc: 76.720% (7672/10000)
Epoch: 11
Loss: 0.595 | Acc: 80.680% (8068/10000)
Saving..
Epoch: 12
Loss: 0.577 | Acc: 81.390% (8139/10000)
Saving..
Epoch: 13
Loss: 0.502 | Acc: 83.340% (8334/10000)
Saving..
Epoch: 14
Loss: 0.593 | Acc: 80.840% (8084/10000)
Epoch: 15
Loss: 0.569 | Acc: 81.670% (8167/10000)
Epoch: 16
Loss: 0.592 | Acc: 80.700% (8070/10000)
Epoch: 17
Loss: 0.577 | 

Loss: 0.198 | Acc: 94.770% (9477/10000)
Epoch: 158
Loss: 0.201 | Acc: 94.690% (9469/10000)
Epoch: 159
Loss: 0.204 | Acc: 94.760% (9476/10000)
Epoch: 160
Loss: 0.204 | Acc: 94.630% (9463/10000)
Epoch: 161
Loss: 0.198 | Acc: 94.840% (9484/10000)
Epoch: 162
Loss: 0.193 | Acc: 94.870% (9487/10000)
Saving..
Epoch: 163
Loss: 0.193 | Acc: 94.910% (9491/10000)
Saving..
Epoch: 164
Loss: 0.193 | Acc: 94.970% (9497/10000)
Saving..
Epoch: 165
Loss: 0.192 | Acc: 94.990% (9499/10000)
Saving..
Epoch: 166
Loss: 0.193 | Acc: 94.940% (9494/10000)
Epoch: 167
Loss: 0.193 | Acc: 94.870% (9487/10000)
Epoch: 168
Loss: 0.192 | Acc: 94.930% (9493/10000)
Epoch: 169
Loss: 0.192 | Acc: 94.960% (9496/10000)
Epoch: 170
Loss: 0.193 | Acc: 94.970% (9497/10000)
Epoch: 171
Loss: 0.194 | Acc: 95.010% (9501/10000)
Saving..
Epoch: 172
Loss: 0.193 | Acc: 94.990% (9499/10000)
Epoch: 173
Loss: 0.194 | Acc: 95.050% (9505/10000)
Saving..
Epoch: 174
Loss: 0.194 | Acc: 95.010% (9501/10000)
Epoch: 175
Loss: 0.193 | Acc: 95.030% (

In [12]:
model_dir = "./checkpoint-no-cross/"
#creating model
# for model_index in range(model_num):
#     print("Processing model %d"%(model_index))
net =wrn(
      num_classes=10,
      depth=16,
      widen_factor=5,
      dropRate=0.3,
    )
net = net.to(device)

#load checkpoint to the model
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    checkpoint = torch.load(model_dir+'ckpt{}.pth'.format(0))
    net.load_state_dict(checkpoint['net'])

net.eval()
    #store the results in a list
#     results=[]
    
correct = 0
total = 0
    
    #vprediction
with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, targets)

        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
#         outputs = net(inputs)#predict using the model
#         outputs = outputs.squeeze(0)
            
#             results.append(outputs.cpu().numpy())#append the current result to the list

#     results = np.array(results)#convert results from tensors to nparray

#     for i in range(len(results)):
#         for j in range(class_num):
#             best_results[i][j] += results[i][j]# adding up the results in order to do soft vote
    acc = 100.*correct/total
    print("Accuracy: %.3f"%(acc))
# best_results = np.argmax(best_results,1)# using soft vote to find the best prediction

# print("finished all")

Accuracy: 95.060
